In [6]:
%matplotlib widget
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import tdt
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import xarray as xr

import kdephys.hypno as kh
import kdephys.pd as kpd
import kdephys.xr as kx
import kdephys.utils as ku
import kdephys.ssfm as ss

import acr.subjects as subs
import acr
import plotly.express as px

In [8]:
p13 = acr.io.get_acr_paths('ACR_13', ['laser1'])
a13 = tdt.read_block(p13['laser1'], evtype=['epocs'])

read from t=0s to t=51460.02s


In [9]:
p14 = acr.io.get_acr_paths('ACR_14', ['laser1'])
a14 = tdt.read_block(p14['laser1'], evtype=['epocs'])

read from t=0s to t=50151.31s


In [11]:
np.timedelta64(51460, 's') + np.datetime64(a13.info.start_date)


numpy.datetime64('2022-09-09T21:23:52.999999')

In [10]:
np.timedelta64(50151, 's') + np.datetime64(a14.info.start_date)


numpy.datetime64('2022-09-12T21:02:31.999999')